<center> <h1> <u> <font color='red'> Hands on Training on Bangla Language Processing Collocated with CLBLP'23 </font> </u> </h1> </center>

In [ ]:
https://gofile.io/d/Mtjk6i

#Libraries

In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

CPU times: user 2.55 s, sys: 410 ms, total: 2.96 s
Wall time: 3.61 s


## Aggresive Dataset


In [ ]:
!wget -O aggression.zip https://www.dropbox.com/s/8h0knnj1fvv4s3t/aggression.zip?dl=0
     

# extract the zip file and save the files into bangla-digit folder
import zipfile
zip_ref = zipfile.ZipFile("aggression.zip", 'r')
zip_ref.extractall("")
zip_ref.close()

--2023-02-21 07:34:07--  https://www.dropbox.com/s/8h0knnj1fvv4s3t/aggression.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/8h0knnj1fvv4s3t/aggression.zip [following]
--2023-02-21 07:34:08--  https://www.dropbox.com/s/raw/8h0knnj1fvv4s3t/aggression.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc31fc47e671168c3ad93d1a0dfb.dl.dropboxusercontent.com/cd/0/inline/B272MikUnszZ6OB1NRCs-75wey7t8PHFuFJqLeZvcNhmnTZodnlp2gBJVj3zLvTn_McNumA8cj4oKtZH93ohvgF3J7HqAbxL8-hik5X99R8ep5prc4yInQkn0ERBI5mx3K34wcYQMxPfjmsV9dPS516GpJMxR_MilXX8fvgjLc1efw/file# [following]
--2023-02-21 07:34:09--  https://uc31fc47e671168c3ad93d1a0dfb.dl.dropboxusercontent.com/cd/0/inline/B272MikUnszZ6OB1NRCs-75wey7t8PHFuFJqLeZvcNhmnTZodnlp2gBJVj3zLvTn_M

In [ ]:
dataset = pd.read_excel("aggression/train-binary.xlsx")
# dataset = pd.read_excel("aggression/train-binary.xlsx")
dataset = dataset.iloc[:,1:]
print(dataset.columns.tolist())
print(dataset[6:10])

['cleaned', 'Class', 'Label']
                                             cleaned Class  Label
6  সুন্দর পরিবেশ তৈরি করতে হবে সিনামা হল নোংরা পর...  NoAG      0
7  আমি এই জায়গা একটি বিশেষ ফ্যান ছিল না কাজ পরে ...  NoAG      0
8  কিন্তু এই পোড়া ভূখন্ডে অমন মানবিক মুগ্ধতা জাগা...  NoAG      0
9  এই কাংকির ছেলে রাজাকার সবাই তো কুত্তালীগ করে এ...    AG      1


In [ ]:
val_dataset = pd.read_excel("aggression/val-multi.xlsx")
print(val_dataset[50:55])

    Unnamed: 0                                            cleaned Class  Label
50          50                      আসলে সরকার পরিবর্তন করা দরকার  PoAG      1
51          51  এটাই হলো মূল কারণ আর সেটা হলো হিন্দু ধর্মে এ ম...  ReAG      0
52          52                 সনাতন ধর্মই নারী কে সম্মান দেয় নাই  ReAG      0
53          53  ঐদিন যদি শেখ হাসিনা মারা যাইতো তাহলে বাংলাদেশ ...  PoAG      1
54          54                           হাসিনার পরিনতি হবে ভয়াবহ  PoAG      1


In [ ]:
test_dataset = pd.read_excel("aggression/test-multi.xlsx")
print(test_dataset[1:4])

   Unnamed: 0                                            cleaned Class  Label
1           1  পৃথিবীতে সর্বপ্রথম কোন দেবতা পুতনা রাক্ষসীর স্...  ReAG      0
2           2  তোর কমেন্টে পাবলিক গালি ছাড়া ভাল কিছু লিখবে না...  VeAG      2
3           3  তাহলে অসুখ করলে ডক্টর না দেখিয়ে আল্লার ইচ্ছে ব...  ReAG      0


In [ ]:
dataset.Class.value_counts()

ReAG    1794
PoAG    1655
VeAG    1629
GeAG     368
Name: Class, dtype: int64

##Evaluation Metrics

In [ ]:

from sklearn.metrics import confusion_matrix,classification_report 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
'''Evaluation Parameters'''

def print_metrices(true,pred):
    print(confusion_matrix(true,pred))
    print(classification_report(true,pred,target_names=['ReAG', 'PoAG', 'VeAG', 'GeAG']))
    print("Accuracy : ",accuracy_score(true,pred))
    print("Precison : ",precision_score(true,pred, average = 'weighted'))
    print("Recall : ",recall_score(true,pred,  average = 'weighted'))
    print("F1 : ",f1_score(true,pred,  average = 'weighted'))

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [ ]:
pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 61.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 64.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.0/266.0 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 49.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━

In [ ]:
import ktrain
from ktrain import text

In [ ]:
X_train = dataset['cleaned'].tolist()
y_train = dataset['Label'].tolist()

X_valid = val_dataset['cleaned'].tolist()
y_valid = val_dataset['Label'].tolist()

X_test = test_dataset['cleaned'].tolist()
#y_test = test_data['Label'].tolist()

In [ ]:
categories = dataset.Class.value_counts().keys()
categories = categories.tolist()
categories

['ReAG', 'PoAG', 'VeAG', 'GeAG']

In [ ]:
model_name = 'bert-base-multilingual-cased'
trans = text.Transformer(model_name,maxlen=50,class_names=categories)

In [ ]:
train = trans.preprocess_train(X_train,y_train)
valid = trans.preprocess_test(X_valid,y_valid)
#test = trans.preprocess_test(X_test,y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 14
	95percentile : 36
	99percentile : 56


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 14
	95percentile : 35
	99percentile : 48


In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=12)

In [ ]:
learner.fit_onecycle(2e-5,1)     #2e-5        



begin training using onecycle policy with max lr of 2e-05...
454/454 [==============================] - 112s 196ms/step - loss: 0.7690 - accuracy: 0.7077 - val_loss: 0.4155 - val_accuracy: 0.8708


In [ ]:
learner.validate(class_names=categories)

22/22 [==============================] - 6s 126ms/step
              precision    recall  f1-score   support

        ReAG       0.90      0.91      0.91       210
        PoAG       0.90      0.95      0.92       229
        VeAG       0.81      0.87      0.84       194
        GeAG       0.88      0.31      0.46        48

    accuracy                           0.87       681
   macro avg       0.87      0.76      0.78       681
weighted avg       0.87      0.87      0.86       681



array([[192,   5,  12,   1],
       [  5, 217,   7,   0],
       [ 13,  11, 169,   1],
       [  4,   8,  21,  15]])

##Save and Load Model

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [ ]:
# Save and load the model
predictor.save('aggressive-bert')

In [ ]:
predictor = ktrain.load_predictor('aggressive-bert')

In [ ]:
y_pred = predictor.predict(X_test)

22/22 [==============================] - 5s 113ms/step


In [ ]:
print_metrices(test_dataset['Class'],y_pred)

[[ 11   4   8  23]
 [  0 185   7   9]
 [  3   6 192  12]
 [  3   5  17 195]]
              precision    recall  f1-score   support

        ReAG       0.65      0.24      0.35        46
        PoAG       0.93      0.92      0.92       201
        VeAG       0.86      0.90      0.88       213
        GeAG       0.82      0.89      0.85       220

    accuracy                           0.86       680
   macro avg       0.81      0.74      0.75       680
weighted avg       0.85      0.86      0.85       680

Accuracy :  0.8573529411764705
Precison :  0.8496456505934886
Recall :  0.8573529411764705
F1 :  0.8464999872455762


## Manual Test

In [ ]:
sample_text = "আমাদের বিপরীত দলের সবগুলারে ধরে পিটানো দরকার"
y_pred = predictor.predict([sample_text])
y_pred

1/1 [==============================] - 0s 134ms/step


['PoAG']

In [ ]:
# load model to generate embeddings
from transformers import *
import tensorflow as tf
tokenizer = AutoTokenizer.from_pretrained(model_name)
input_ids = tf.constant(tokenizer.encode(sample_text))[None, :]  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
print(last_hidden_states.numpy())  # print shape of embedding vectors

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size":

[[-1.3672739  2.9301293 -0.6274542 -1.0523622]]


##Reference


1. [Text Classification Using Transformers](https://nbviewer.jupyter.org/github/amaiya/ktrain/blob/master/tutorials/tutorial-A3-hugging_face_transformers.ipynb)

2. [Related Article](https://towardsdatascience.com/text-classification-with-hugging-face-transformers-in-tensorflow-2-without-tears-ee50e4f3e7ed)

3. [Ktrain](https://github.com/amaiya/ktrain)

4. [m-BERT](https://huggingface.co/bert-base-multilingual-cased)

5. [Text Embedding](https://www.baeldung.com/cs/transformer-text-embeddings)